# Week 4

Yay! It's week 4. Today's we'll keep things light. 

I've noticed that many of you are struggling a bit to keep up and still working on exercises from the previous weeks. Thus, this week we only have two components with no lectures and very little reading. 


## Overview

* An exercise on visualizing geodata using a different set of tools from the ones we played with during Lecture 2.
* Thinking about visualization, data quality, and binning. Why ***looking at the details of the data before applying fancy methods*** is often important.

## Part 1: Visualizing geo-data

It turns out that `plotly` (which we used during Week 2) is not the only way of working with geo-data. There are many different ways to go about it. (The hard-core PhD and PostDoc researchers in my group simply use matplotlib, since that provides more control. For an example of that kind of thing, check out [this one](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621).)

Today, we'll try another library for geodata called "[Folium](https://github.com/python-visualization/folium)". It's good for you all to try out a few different libraries - remember that data visualization and analysis in Python is all about the ability to use many different tools. 

The exercise below is based on the code illustrated in this nice [tutorial](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data), so let us start by taking a look at that one.

*Reading*. Read through the following tutorial
 * "How to: Folium for maps, heatmaps & time data". Get it here: https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data
 * (Optional) There are also some nice tricks in "Spatial Visualizations and Analysis in Python with Folium". Read it here: https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf

> *Exercise 1.1*: A new take on geospatial data. 
>
>A couple of weeks ago (Part 3 of Week 2), we worked with spacial data by using color-intensity of shapefiles to show the counts of certain crimes within those individual areas. Today, we look at studying geospatial data by plotting raw data points as well as heatmaps on top of actual maps.
> 
> * First start by plotting a map of San Francisco with a nice tight zoom. Simply use the command `folium.Map([lat, lon], zoom_start=13)`, where you'll have to look up San Francisco's longitude and latitude.
> * Next, use the the coordinates for SF City Hall `37.77919, -122.41914` to indicate its location on the map with a nice, pop-up enabled maker. (In the screenshot below, I used the black & white Stamen tiles, because they look cool).
> <img src="https://raw.githubusercontent.com/suneman/socialdata2022/main/files/city_hall_2022.png" alt="drawing" width="600"/>
> * Now, let's plot some more data (no need for pop-ups this time). Select a couple of months of data for `'DRUG/NARCOTIC'` and draw a little dot for each arrest for those two months. You could, for example, choose June-July 2016, but you can choose anything you like - the main concern is to not have too many points as this uses a lot of memory and makes Folium behave non-optimally. 
> We can call this kind of visualization a *point scatter plot*.

In [3]:
# 3.1 import and preprocessing 
import pandas as pd
import numpy as np
from pathlib import Path

# read police incidence reports, historical 2003 to may 2018
police_incidence_reports = pd.read_csv(Path.cwd().parent/"files"/"Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")
police_incidence_reports = pd.DataFrame(police_incidence_reports)
police_incidence_reports_2003_till_2017 = police_incidence_reports[~(pd.DatetimeIndex(police_incidence_reports['Date']).year > 2017)]
police_incidence_reports_2003_till_2017['Datetime'] = pd.to_datetime(police_incidence_reports_2003_till_2017['Date']+ ' ' +\
    police_incidence_reports_2003_till_2017['Time'], format="%m/%d/%Y %H:%M")
police_incidence_reports_2003_till_2017 = police_incidence_reports_2003_till_2017.sort_values(['Category', 'Datetime']).reset_index()

C:\Users\phili\AppData\Local\Temp\ipykernel_2464\2325216064.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  police_incidence_reports_2003_till_2017['Datetime'] = pd.to_datetime(police_incidence_reports_2003_till_2017['Date']+ ' ' +\


In [4]:
import folium

lat_SF = 37.773972
lon_SF = -122.431297

map_SF = folium.Map(location=[lat_SF, lon_SF], tiles = "Stamen Toner", zoom_start=13)
folium.Marker([37.77919, -122.41914], popup='SF City Hall').add_to(map_SF)

drug_narcotic = police_incidence_reports_2003_till_2017.loc[police_incidence_reports_2003_till_2017['Category'].isin(['DRUG/NARCOTIC'])]\
    .sort_values(['Datetime']).reset_index()
drug_narcotic_June_till_July_2016 = drug_narcotic.loc[(drug_narcotic['Datetime'] >= '01-06-2016 00:00:00') &\
    (drug_narcotic['Datetime'] <= '31-07-2016 23:59:59')]

for ind in drug_narcotic_June_till_July_2016.index:
    folium.Marker([drug_narcotic_June_till_July_2016['Y'][ind], drug_narcotic_June_till_July_2016['X'][ind]], \
        popup=drug_narcotic_June_till_July_2016['PdId'][ind]).add_to(map_SF)

map_SF





Ok. Time for a little break. Note that a nice thing about Folium is that you can zoom in and out of the maps.

> *Exercise 1.2*: Heatmaps.
> * Now, let's play with **heatmaps**. You can figure out the appropriate commands by grabbing code from the main [tutorial](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data)) and modifying to suit your needs.
>    * To create your first heatmap, grab all arrests for the category `'SEX OFFENSES, NON FORCIBLE'` across all time. Play with parameters to get plots you like.
>    * Now, comment on the differences between scatter plots and heatmaps. 
>.      - What can you see using the scatter-plots that you can't see using the heatmaps? 
>.      - And *vice versa*: what does the heatmaps help you see that's difficult to distinguish in the scatter-plots?
>    * Play around with the various parameters for heatmaps. You can find a list here: https://python-visualization.github.io/folium/plugins.html
>    * Comment on the effect on the various parameters for the heatmaps. How do they change the picture? (at least talk about the `radius` and `blur`).
> For one combination of settings, my heatmap plot looks like this.
> <img src="https://raw.githubusercontent.com/suneman/socialdata2022/main/files/crime_hot_spot.png" alt="drawing" width="600"/>
>    * In that screenshot, I've (manually) highlighted a specific hotspot for this type of crime. Use your detective skills to find out what's going on in that building on the 800 block of Bryant street ... and explain in your own words. 

(*Fun fact*: I remembered the concentration of crime-counts discussed at the end of this exercise from when I did the course back in 2016. It popped up when I used a completely different framework for visualizing geodata called [`geoplotlib`](https://github.com/andrea-cuttone/geoplotlib). You can spot it if you go to that year's [lecture 2](https://nbviewer.jupyter.org/github/suneman/socialdataanalysis2016/blob/master/lectures/Week3.ipynb), exercise 4.)

In [5]:
from folium.plugins import HeatMap

sex_offenses_non_forcible = police_incidence_reports_2003_till_2017.loc[police_incidence_reports_2003_till_2017['Category'].\
    isin(['SEX OFFENSES, NON FORCIBLE'])].sort_values(['Datetime']).reset_index()

map_SF = folium.Map(location=[lat_SF, lon_SF], tiles = "Stamen Toner", zoom_start=13)

heat_data = [[row['Y'],row['X']] for index, row in sex_offenses_non_forcible.iterrows()]

HeatMap(heat_data, radius = 15, blur = 3).add_to(map_SF)

map_SF

#difference scatterplot and heatmap
# https://chartio.com/learn/charts/heatmap-complete-guide/#:~:text=Scatter%20plot,variables%2C%20an%20issue%20called%20overplotting.

# radius of the points get bigger -> more overlapping -> more spots with different colors
# blur -> the transitions are more or less blurred

# 800th block of Bryant street -> people argue due to the large homeless shelter 

For the final element of working with heatmaps, let's now use the cool Folium functionality `HeatMapWithTime` to create a visualization of how the patterns of your favorite crime-type changes over time.

> *Exercise 1.3*: Heatmap movies. This exercise is a bit more independent than above - you get to make all the choices.
> * Start by choosing your favorite crimetype. Prefereably one with spatial patterns that change over time (use your data-exploration from the previous lectures to choose a good one).
> * Now, choose a time-resolution. You could plot daily, weekly, monthly datasets to plot in your movie. Again the goal is to find interesting temporal patterns to display. We want at least 20 frames though.
> * Create the movie using `HeatMapWithTime`.
> * Comment on your results: 
>   - What patterns does your movie reveal?
>   - Motivate/explain the reasoning behind your choice of crimetype and time-resolution. 

In [ ]:
#HeatMap with Time
from folium import plugins

drug_narcotic = police_incidence_reports_2003_till_2017.loc[police_incidence_reports_2003_till_2017['Category'].isin(['DRUG/NARCOTIC'])]\
    .sort_values(['Datetime']).reset_index()
drug_narcotic_August_2016 = drug_narcotic.loc[(drug_narcotic['Datetime'] >= '01-08-2016 00:00:00') &\
    (drug_narcotic['Datetime'] <= '31-08-2016 23:59:59')]

ind = list(range(1,32))
map4 = folium.Map(location=[lat_SF, lon_SF],tiles = 'CartoDB Positron',zoom_start=9,width=800,height=800,control_scale=True)
#heat_data2 is a List of Lists of Lists
heat_data2 = [ [ [row['Y'],row['X'] ] for index, row in drug_narcotic_August_2016[drug_narcotic_August_2016['Datetime'].\
    apply(lambda x: x.day==j)].iterrows() ] for j in ind ]


plugins.HeatMapWithTime(heat_data2,index=ind).add_to(map4)
plugins.Fullscreen().add_to(map4)


map4

# most of the crime in the innercity, but also temporary hotspots spread around during the month, also just reoccuring places


## Part 2: Errors in the data. The importance of looking at raw (or close to raw) data.

We started the course by plotting simple histogram and bar plots that showed a lot of cool patterns. But sometimes the binning can hide imprecision, irregularity, and simple errors in the data that could be misleading. In the work we've done so far, we've already come across at least three examples of this in the SF data. 

1. In the temporal activity for `PROSTITUTION` something surprising is going on on Thursday. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/prostitution.png), where I've highlighted the phenomenon I'm talking about.
2. When we investigated the details of how the timestamps are recorded using jitter-plots, we saw that many more crimes were recorded e.g. on the hour, 15 minutes past the hour, and to a lesser in whole increments of 10 minutes. Crimes didn't appear to be recorded as frequently in between those round numbers. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/jitter.png), where I've highlighted the phenomenon I'm talking about.
3. And, today we saw that the Hall of Justice seemed to be an unlikely hotspot for sex offences. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/crime_hot_spot.png).

> *Exercise 2*: Data errors. The data errors we discovered above become difficult to notice when we aggregate data (and when we calculate mean values, as well as statistics more generally). Thus, when we visualize, errors become difficult to notice when binning the data. We explore this process in the exercise below.
>
>This last exercise for today has two parts:
> * In each of the examples above, describe in your own words how the data-errors I call attention to above can bias the binned versions of the data. Also, briefly mention how not noticing these errors can result in misconceptions about the underlying patterns of what's going on in San Francisco (and our modeling).
> * Find your own example of human noise in the data and visualize it.

1. "I would say it is surprising to have a much bigger amount of prostitution recorded in a specific moment on Thursday than on every other moment in time. The spike that you see in the plot I've linked in Week 4 seems to be an outlier." quote Anna -> there is one minute where over 120 times prostitution was recorded -> usually 50 is already a lot

2. Probably the police people just do a rough estimate when they fill out the paper work regarding the crime. -> there is bias on these specific times if you would like to e.g. predict when the next crime could happen

3. probably show wrong spot 

In [ ]:
# Exercise 2.1 -> tried to recreate the plot
# Prostitution on Thursday of Week 2 in a 15 min resolution

police_incidence_reports['Datetime'] = pd.to_datetime(police_incidence_reports['Date']+ ' ' +\
    police_incidence_reports['Time'], format="%m/%d/%Y %H:%M")

prostitution = police_incidence_reports.loc[police_incidence_reports['Category'].isin(['PROSTITUTION'])]\
    .sort_values(['Datetime']).reset_index()

prostitution_15_min_resolution = prostitution.groupby(pd.Grouper(key="Datetime", freq="15Min"))['PdId'].count()
prostitution_15_min_resolution = pd.DataFrame({'Datetime_15min':prostitution_15_min_resolution.index, \
    'Count_15min':prostitution_15_min_resolution.values})

prostitution_15_min_resolution['Hour_of_the_Week'] = prostitution_15_min_resolution['Datetime_15min'].dt.dayofweek * 24 + \
    (prostitution_15_min_resolution['Datetime_15min'].dt.hour + 1)

prostitution_15_min_resolution_hour_of_week = prostitution_15_min_resolution.groupby(['Hour_of_the_Week']).agg({'Count_15min':'sum'})

import matplotlib.pyplot as plt

plt.bar(x = prostitution_15_min_resolution_hour_of_week.index , height = prostitution_15_min_resolution_hour_of_week['Count_15min'])


In [ ]:
# police_incidence_reports_2003_till_2017.info()
# police_incidence_reports_2003_till_2017.describe()

# some weird lat and lon -> e.g. [Y, X] = [Y = 90, X = -120.5]

# X -> lon
# Y -> lat

test = police_incidence_reports_2003_till_2017[police_incidence_reports_2003_till_2017['X'] > -121]

lat_SF = 37.773972
lon_SF = -122.431297

map_SF = folium.Map(location=[lat_SF, lon_SF], tiles = "Stamen Toner", zoom_start=13)
folium.Marker([90, -120.5], popup='Test').add_to(map_SF)

map_SF